In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


model_name = "roberta-base" # try: "bert-base-uncased", "albert-base-v2", "microsoft/deberta-v3-base"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


ds = load_dataset("imdb")


def tok(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)


encoded = ds.map(tok, batched=True)
encoded = encoded.rename_column("label", "labels")
encoded.set_format("torch", columns=["input_ids","attention_mask","labels"])


args = TrainingArguments(
"roberta-imdb",
learning_rate=2e-5, num_train_epochs=3, per_device_train_batch_size=16,
per_device_eval_batch_size=32, weight_decay=0.01, eval_strategy="epoch",
fp16=True,
report_to="none"
)


def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
    "acc": accuracy_score(p.label_ids, preds),
    "f1": f1_score(p.label_ids, preds, average="macro"),
    "prec": precision_score(p.label_ids, preds, average="macro"),
    "rec": recall_score(p.label_ids, preds, average="macro"),
    }


trainer = Trainer(model=model, args=args, train_dataset=encoded["train"], eval_dataset=encoded["test"], compute_metrics=compute_metrics)
trainer.train()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
